#### load dependencies

In [1]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import os
from keras.datasets import imdb
%matplotlib inline

C:\Users\Munna chowhan\Anaconda3\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### set hyperparameters

In [2]:
output_dir = "model_output/Vanilla_LSTM/"

epochs = 4
batch_size = 128

#embedding layer:
n_dims = 64
max_review_length = 100
n_unique_words = 10000
#while padding if the review is < 100 words pad it at the beginning of the review.
#while truncating if the review is > 100 cut the review at the beginning
pad_type = trunc_type = "pre"
dropout_embed = 0.2
#conv layer:
n_lstm = 256
dropout_lstm = 0.2


#dense_layer:
#n_dense = 256
#dropout = 0.2

####  load data

In [3]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=n_unique_words)

### preprocess data

In [4]:
x_train = pad_sequences(x_train,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)
x_test = pad_sequences(x_test,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)

### model architecture

In [6]:
model = Sequential()
model.add(Embedding(n_unique_words,n_dims,input_length=max_review_length))
model.add(SpatialDropout1D(dropout_embed))
model.add(LSTM(units=n_lstm,dropout=dropout_lstm))
model.add(Dense(1, activation="sigmoid"))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               328704    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 968,961
Trainable params: 968,961
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])

#### save the model weights for  every epoch to load the model with high accuracy


In [9]:
modelcheckpoint  = ModelCheckpoint(filepath=output_dir +"weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### train the model

In [10]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test),verbose=1, callbacks=[modelcheckpoint])

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 264s 11ms/step - loss: 0.4813 - acc: 0.7527 - val_loss: 0.3435 - val_acc: 0.8490
Epoch 2/4
25000/25000 [==============================] - 238s 10ms/step - loss: 0.2945 - acc: 0.8815 - val_loss: 0.3422 - val_acc: 0.8508
Epoch 3/4
25000/25000 [==============================] - 238s 10ms/step - loss: 0.2406 - acc: 0.9067 - val_loss: 0.3845 - val_acc: 0.8416
Epoch 4/4
25000/25000 [==============================] - 263s 11ms/step - loss: 0.2052 - acc: 0.9223 - val_loss: 0.3623 - val_acc: 0.8445


#### load model weights with high validation accurcy

In [ ]:
model.load_weights(filepath=output_dir+"/weights.01.hdf5")

#### make predictions on test set

In [ ]:
y_pred = model.predict_proba(x_test)

In [ ]:
y_pred[0]

In [ ]:
plt.hist(y_pred)
_ = plt.axvline(x = 0.5, color="red")

In [ ]:
auc = roc_auc_score(y_test,y_pred)*100.0

In [ ]:
auc

In [ ]:
predicted_sentiment = []
for i in y_pred:
    predicted_sentiment.append(i[0])

In [ ]:
df = pd.DataFrame(list(zip(predicted_sentiment,y_test)),columns=["predicted","real"])

In [ ]:
df.head()